# Practical introduction to *Prompt Engineering* and the impact of inference parameters

Objectives:
- Handle *prompt engineering* techniques (clear instructions, role/persona, constraints, *few-shot*, structured format, self-checking).
- Experiment with the effect of inference parameters (temperature, top_p, top_k, penalties, max_tokens, seed) on variability, accuracy and style.
- Set up a small loop for evaluating and plotting results.

> ⚠️ **Keys/API and models**: by default, this tutorial uses the API of a compatible provider (e.g. OpenAI) via the environment variable `OPENAI_API_KEY`.  
> You can use any *chat/completions* model (e.g. `gpt-4o-mini`, `gpt-4.1`, `o4-mini`...) or adapt the `LLMClient` class for another provider (OpenRouter, vLLM, etc.).

## Lesson plan
1. Preparing the environment and API
2. Basic *prompting* techniques (instructions, roles, constraints)
3. *Few-shot prompting
4. Structured output (JSON + validation)

## 1) Preparing the environment and LLM client

In [1]:
import os, time, random, statistics, json
from dataclasses import dataclass
from typing import List, Dict, Any, Optional, Tuple
from jsonschema import validate, ValidationError
import requests
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

@dataclass
class GenParams:
    model: str = "gpt-4o-mini"   # par défaut côté OpenAI; côté Ollama on substituera si nécessaire
    temperature: float = 0.7
    top_p: float = 1.0
    max_tokens: int = 300
    frequency_penalty: float = 0.0
    presence_penalty: float = 0.0
    seed: Optional[int] = None   # supporté par Ollama via options.seed et par OpenAI (selon modèles)


class LLMClient:
    """
    Client LLM bi-backend:
      - OpenAI via SDK et clé API
      - Ollama via HTTP (déjà lancé avec `ollama serve`)
    Sélection via la variable d'env USE_OLLAMA=1|true|yes
    """

    use_ollama = False
    ollama_base = "http://localhost:11434"

    def __init__(self):
        if self.use_ollama:
            print(f"⚙️ Backend: Ollama ({self.ollama_base})")
            # test rapide de santé (facultatif, mais pratique)
            try:
                r = requests.get(f"{self.ollama_base}/api/tags", timeout=3)
                r.raise_for_status()
            except Exception as e:
                raise RuntimeError(
                    f"Ollama semble indisponible sur {self.ollama_base}. "
                    f"Assurez-vous que `ollama serve` tourne. Détails: {e}"
                )
            self.client = None
        else:
            print("⚙️ Backend: OpenAI")
            api_key = os.environ.get("OPENAI_API_KEY")
            if not api_key:
                raise RuntimeError(f"Renseignez votre clé API dans {api_key}.")
            if OpenAI is None:
                raise RuntimeError("Le paquet 'openai' n'est pas installé. Faites: pip install --upgrade openai")
            self.client = OpenAI(api_key=api_key)

    # ---------- Implémentation Ollama (HTTP) ----------
    def _ensure_ollama_model(self, model: str) -> str:
        """
        Si l'utilisateur a laissé un modèle OpenAI (ex: gpt-4o-mini) alors qu'on est en mode Ollama,
        on substitue un modèle local raisonnable.
        """
        if not model or model.startswith("gpt-"):
            return "mistral"  # à ajuster selon vos modèles téléchargés: e.g. "qwen2.5:7b"
        return model

    def _chat_ollama(self, system: str, user: str, params: GenParams) -> str:
        url = f"{self.ollama_base}/api/chat"
        model = self._ensure_ollama_model(params.model)

        messages = []
        if system:
            messages.append({"role": "system", "content": system})
        messages.append({"role": "user", "content": user})

        payload = {
            "model": model,
            "messages": messages,
            "stream": False,
            # Mapping des options vers Ollama
            "options": {
                # temperature / top_p / seed sont supportés par Ollama
                "temperature": params.temperature,
                "top_p": params.top_p,
                **({"seed": params.seed} if params.seed is not None else {}),
                # Ollama n'a pas `max_tokens`, mais `num_predict`
                "num_predict": params.max_tokens if params.max_tokens is not None else -1,
                # Pas d'équivalents directs pour presence/frequency_penalty
            },
        }

        try:
            r = requests.post(url, json=payload, timeout=120)
            r.raise_for_status()
            data = r.json()
            # Réponse typique: {"message": {"role":"assistant","content":"..."} , ...}
            msg = data.get("message", {})
            content = msg.get("content", "")
            return content.strip()
        except requests.HTTPError as e:
            try:
                err = r.json()
            except Exception:
                err = r.text
            raise RuntimeError(f"Erreur HTTP Ollama: {e} | {err}")
        except Exception as e:
            raise RuntimeError(f"Echec appel Ollama: {e}")

    # ---------- Implémentation OpenAI (SDK) ----------
    def _chat_openai(self, system: str, user: str, params: GenParams) -> str:
        messages = []
        if system:
            messages.append({"role": "system", "content": system})
        messages.append({"role": "user", "content": user})

        rsp = self.client.chat.completions.create(
            model=params.model,
            messages=messages,
            temperature=params.temperature,
            top_p=params.top_p,
            max_tokens=params.max_tokens,
            frequency_penalty=params.frequency_penalty,
            presence_penalty=params.presence_penalty,
            seed=params.seed,
        )
        return rsp.choices[0].message.content

    # ---------- API publique ----------
    def chat(self, system: str, user: str, params: GenParams) -> str:
        if self.use_ollama:
            return self._chat_ollama(system, user, params)
        return self._chat_openai(system, user, params)


# ---------- Paramètres par défaut ----------
DEFAULT_PARAMS = GenParams(
    model="gpt-4.1-nano",  # si USE_OLLAMA=1, on substituera automatiquement par "llama3.1"
    temperature=0.7,
    top_p=1.0,
    max_tokens=300,
)

client = LLMClient()
print("Client prêt.")

⚙️ Backend: OpenAI
Client prêt.


## 2) Basic *prompting* techniques

We're going to compare several variants for the same simple task (e.g. summarizing a short article).  
Experiment: **without instructions**, **with constraints**, **with role/persona**.

**Exercise:** Fill in the cells to formulate different prompts and compare the outputs.

In [2]:
task_text = (    "Le *prompt engineering* consiste à concevoir des instructions claires pour guider un modèle de langage. "
    "Il inclut des techniques comme le rôle, les contraintes de format, le few-shot, et la calibration des paramètres d'inférence. "
    "Son objectif est d'améliorer la fiabilité, la structure et l'utilité des réponses."
)

### 2.a No instruction (baseline)

In [3]:
p_no_instruction = "Peux-tu m'aider avec ce texte ?"
out_no_instruction = client.chat(system="", user=f"{p_no_instruction}\n\nTexte: {task_text}", params=DEFAULT_PARAMS)
print("=== Baseline ===\n", out_no_instruction, "\n")

=== Baseline ===
 Bien sûr ! Voici une version légèrement reformulée de ton texte pour plus de clarté et de fluidité :

Le *prompt engineering* consiste à élaborer des instructions précises afin de guider efficacement un modèle de langage. Cela inclut des techniques telles que l'attribution de rôles, l'imposition de contraintes de format, l'utilisation du few-shot learning, ainsi que la calibration des paramètres d'inférence. L'objectif est d'améliorer la fiabilité, la cohérence et l'utilité des réponses générées par le modèle. 



### 2.b Clear instructions + measurable objectives

In [4]:
p_instruction = """
Résume le texte en **2 phrases maximum**, en français.
N'utilise **aucune** liste à puces.
Ajoute une phrase de conclusion commençant par "En bref,".
"""
out_instruction = client.chat(system="", user=f"{p_instruction}\n\nTexte: {task_text}", params=DEFAULT_PARAMS)
print("=== Instruction claire ===\n", out_instruction, "\n")

=== Instruction claire ===
 Le *prompt engineering* vise à créer des instructions précises pour optimiser les réponses des modèles de langage en utilisant différentes techniques. En bref, cette pratique permet d'améliorer la qualité, la cohérence et la pertinence des résultats générés par l'intelligence artificielle. 



### 2.c Role / persona + constraints

In [5]:
p_role = """
Vous êtes un assistant pédagogique concis et rigoureux.
Ton: neutre, précis.
Tâche: reformule le texte en **150 mots max** et ajoute un **TITRE EN MAJUSCULES** en première ligne.
"""
out_role = client.chat(system="", user=f"{p_role}\n\nTexte: {task_text}", params=DEFAULT_PARAMS)
print("=== Persona + contraintes ===\n", out_role, "\n")

=== Persona + contraintes ===
 PROMPT ENGINEERING : OPTIMISATION DES INSTRUCTIONS POUR LES MODÈLES DE LANGAGE

Le *prompt engineering* désigne l'art de créer des instructions précises afin d'orienter efficacement un modèle de langage. Il intègre diverses techniques telles que l'attribution d’un rôle spécifique, l'imposition de contraintes de format, l’utilisation de l’approche few-shot, et la calibration des paramètres d’inférence. Ces méthodes visent à renforcer la fiabilité, à structurer les réponses, et à maximiser leur utilité. En élaborant des prompts soigneusement conçus, l’utilisateur peut guider le modèle pour qu’il produise des résultats plus pertinents et cohérents. Cette discipline est essentielle pour exploiter pleinement le potentiel des modèles de langage, notamment dans des applications variées telles que l’assistance, la génération de contenu ou l’analyse de données. En résumé, le prompt engineering permet d’optimiser la communication avec l’intelligence artificielle en

## 3) *Few-shot prompting*

Showing the model 2-3 examples of inputs/outputs can guide it towards the expected style.

**Exercise:** create 2 examples of *paraphrase* and ask the model to produce a third.

In [6]:
fewshot_examples = [
    {"input": "Le chat dort sur le canapé.", "output": "Le félin sommeille sur le sofa."},
    {"input": "Ce modèle fait des erreurs occasionnelles.", "output": "Ce système commet parfois des inexactitudes."},
]

instruction = "Paraphrase la phrase en conservant le sens et un ton neutre."

def build_fewshot_prompt(examples, new_input):
    lines = [instruction, "", "Exemples:"]
    for ex in examples:
        lines.append(f"- Input: {ex['input']}")
        lines.append(f"  Output: {ex['output']}")
    lines.append("")
    lines.append(f"Nouvelle phrase: {new_input}")
    lines.append("Donne uniquement la paraphrase en français.")
    return "\n".join(lines)

user_input = "L'apprentissage par renforcement utilise des récompenses pour guider l'agent."
prompt = build_fewshot_prompt(fewshot_examples, user_input)
out_fewshot = client.chat(system="", user=prompt, params=DEFAULT_PARAMS)
print(out_fewshot)

L'apprentissage par renforcement emploie des récompenses pour orienter l'agent.


In [7]:
from pprint import pprint
pprint(prompt)

('Paraphrase la phrase en conservant le sens et un ton neutre.\n'
 '\n'
 'Exemples:\n'
 '- Input: Le chat dort sur le canapé.\n'
 '  Output: Le félin sommeille sur le sofa.\n'
 '- Input: Ce modèle fait des erreurs occasionnelles.\n'
 '  Output: Ce système commet parfois des inexactitudes.\n'
 '\n'
 "Nouvelle phrase: L'apprentissage par renforcement utilise des récompenses "
 "pour guider l'agent.\n"
 'Donne uniquement la paraphrase en français.')


## 4) Structured output (JSON) + validation

**Exercise:** force a **JSON** output respecting a minimal schema and validate it automatically.

In [10]:
prompt_json = """
Produis une synthèse **au format JSON strict** du texte ci-dessous.
Champs requis:
- "titre": string
- "mots_cles": array de 3 à 5 strings
- "resume": string (<= 60 mots)
Réponds **uniquement** par un JSON valide, sans texte en dehors du JSON.

Texte:
La classification supervisée apprend une fonction qui associe des entrées à des étiquettes à partir d'exemples annotés.
"""

raw = client.chat(system="", user=prompt_json, params=DEFAULT_PARAMS)
print(raw)

{
  "titre": "Classification supervisée",
  "mots_cles": ["apprentissage automatique", "classification", "étiquettes", "données annotées", "modèle"],
  "resume": "La classification supervisée consiste à apprendre une fonction associant des entrées à des étiquettes à partir d'exemples annotés." 
}


### Validation

In [11]:
schema = {
    "type": "object",
    "properties": {
        "titre": {"type": "string"},
        "mots_cles": {"type": "array", "items": {"type": "string"}},
        "resume": {"type": "string"}
    },
    "required": ["titre", "mots_cles", "resume"]
}

try:
    data = json.loads(raw)
    validate(data, schema)
    print("JSON valide. Titre:", data["titre"])
except (json.JSONDecodeError, ValidationError) as e:
    print("Sortie invalide:", e)

JSON valide. Titre: Classification supervisée
